<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Implementations/blob/master/Transformer/Portuguese%20to%20English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Author: Satwik Ram K
Portuguese to English Transalation

## Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
import math
import re
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


## Loading Dataset

The dataset is available in Tfds

In [2]:
data, metadata = tfds.load('ted_hrlr_translate/pt_to_en',with_info = True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompletePVWXQB/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompletePVWXQB/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompletePVWXQB/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [3]:
data

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>,
 'validation': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>}

In [4]:
metadata

tfds.core.DatasetInfo(
    name='ted_hrlr_translate',
    version=1.0.0,
    description='Data sets derived from TED talk transcripts for comparing similar language pairs
where one is high resource and the other is low resource.
',
    homepage='https://github.com/neulab/word-embeddings-for-nmt',
    features=Translation({
        'en': Text(shape=(), dtype=tf.string),
        'pt': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=54781,
    splits={
        'test': 1803,
        'train': 51785,
        'validation': 1193,
    },
    supervised_keys=('pt', 'en'),
    citation="""@inproceedings{Ye2018WordEmbeddings,
      author  = {Ye, Qi and Devendra, Sachan and Matthieu, Felix and Sarguna, Padmanabhan and Graham, Neubig},
      title   = {When and Why are pre-trained word embeddings useful for Neural Machine Translation},
      booktitle = {HLT-NAACL},
      year    = {2018},
      }""",
    redistribution_info=,
)

## Taking Train and Validation Data

In [5]:
train, val = data['train'], data['validation']

In [6]:
type(train)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

## Creating a custom subwords tokenizer from the training dataset.

In [7]:
count = 0
for a, b in train:
  count += 1
  if count == 10:
    break
  else:
    print("Portugees sentence:",a)
    print("English Corresponding Sentence",b)
    print("--"*60)
  

Portugees sentence: tf.Tensor(b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .', shape=(), dtype=string)
English Corresponding Sentence tf.Tensor(b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .', shape=(), dtype=string)
------------------------------------------------------------------------------------------------------------------------
Portugees sentence: tf.Tensor(b'mas e se estes fatores fossem ativos ?', shape=(), dtype=string)
English Corresponding Sentence tf.Tensor(b'but what if it were active ?', shape=(), dtype=string)
------------------------------------------------------------------------------------------------------------------------
Portugees sentence: tf.Tensor(b'mas eles n\xc3\xa3o tinham a curiosidade de me testar .', shape=(), dtype=string)
English Corresponding Sentence tf.Tensor(b"but they did n't test for curiosity .", shape=(), 

In [8]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train), target_vocab_size = 2**13
)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train), target_vocab_size = 2**13
)

In [9]:
tokenizer_en.save_to_file('tokenizer_en')
tokenizer_pt.save_to_file('tokenizer_pt')

In [10]:
# Loading saved tokenizer
encoder_en = tfds.features.text.SubwordTextEncoder.load_from_file('/content/tokenizer_en')
encoder_pt = tfds.features.text.SubwordTextEncoder.load_from_file('/content/tokenizer_pt')

In [11]:
sample_string = "Transfomer is cool."

encoded_string = encoder_en.encode(sample_string)

original_string = encoder_en.decode(encoded_string)

print("Orginal String:",sample_string)
print("Encoded String:",encoded_string)
print("Decoded String:",original_string)

Orginal String: Transfomer is cool.
Encoded String: [7915, 1248, 7946, 1391, 2762, 7863, 13, 1671, 7877]
Decoded String: Transfomer is cool.


In [12]:
for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder_en.decode([ts])))

7915 ----> T
1248 ----> ran
7946 ----> s
1391 ----> fo
2762 ----> mer
7863 ---->  
13 ----> is 
1671 ----> cool
7877 ----> .


In [13]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

## Add a start and end token to the input and target.

In [14]:
print(encoder_pt.vocab_size)
print(encoder_en.vocab_size)

8214
8087


In [15]:
a = [encoder_pt.vocab_size]
b = np.array([2])
print(b[0])
c = a + b[0]
print(c)

2
[8216]


In [16]:
def encode(lang1, lang2):
  lang1 = [encoder_pt.vocab_size] + encoder_pt.encode(lang1.numpy()) + [encoder_pt.vocab_size + 1]

  lang2 = [encoder_en.vocab_size] + encoder_en.encode(lang2.numpy()) + [encoder_en.vocab_size + 1]

  return lang1, lang2

In [17]:
def tf_encode(pt, en):
  result_pt, result_en = tf.py_function(encode, [pt, en], [tf.int64, tf.int64])
  result_pt.set_shape([None])
  result_en.set_shape([None])

  return result_pt, result_en

## Dropping long sentences over 40 tokens 

In [18]:
MAX_LENGTH = 40

In [19]:
def filter_max_length(x, y, max_length = MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length, tf.size(y) <= max_length)

In [20]:
train_dataset = train.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)

# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE)

In [21]:
pt_batch, en_batch = next(iter(val_dataset))
pt_batch, en_batch

(<tf.Tensor: shape=(64, 38), dtype=int64, numpy=
 array([[8214,  342, 3032, ...,    0,    0,    0],
        [8214,   95,  198, ...,    0,    0,    0],
        [8214, 4479, 7990, ...,    0,    0,    0],
        ...,
        [8214,  584,   12, ...,    0,    0,    0],
        [8214,   59, 1548, ...,    0,    0,    0],
        [8214,  118,   34, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(64, 40), dtype=int64, numpy=
 array([[8087,   98,   25, ...,    0,    0,    0],
        [8087,   12,   20, ...,    0,    0,    0],
        [8087,   12, 5453, ...,    0,    0,    0],
        ...,
        [8087,   18, 2059, ...,    0,    0,    0],
        [8087,   16, 1436, ...,    0,    0,    0],
        [8087,   15,   57, ...,    0,    0,    0]])>)

## Positional Encoding

In [22]:
def get_angles(pos, i, d_model):
  angles_rates = 1 / np.power(1000, (2 * (i//2)) / np.float32(d_model))
  return pos * angles_rates

np.newaxis will expand the dimension of the array

In [23]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [24]:
pos_encoding = positional_encoding(50, 512)
print(pos_encoding.shape)

(1, 50, 512)


## Masking

Mask all the pad tokens in the batch of sequence. It ensures that the model does not treat padding as the input. The mask indicates where pad value 0 is present: it outputs a 1 at those locations, and a 0 otherwise.

In [25]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.

  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [26]:
xx = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(xx)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 1., 1., 0.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[1., 1., 1., 0., 0.]]]], dtype=float32)>

The look-ahead mask is used to mask the future tokens in a sequence. In other words, the mask indicates which entries should not be used.

In [27]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [28]:
xx = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(xx.shape[1])
temp  

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 1.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

## Scaled Dot Product attention

In [29]:
def scaled_dot_product_attention(q, k, v, mask):

  matmul_qk = tf.matmul(q, v, transpose_b = True)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # Applying softmax
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)

  output = tf.matmul(attention_weights, v)

  return output, attention_weights

In [30]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('Attention weights are:')
  print (temp_attn)
  print ('Output is:')
  print (temp_out)

## Multi-head Attention

In [31]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    assert d_model % num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)


  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)
    k = self.wk(k)
    v = self.wv(v)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model)) 
    
    output = self.dense(concat_attention)

    return output, attention_weights

In [32]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

(TensorShape([1, 60, 512]), TensorShape([1, 8, 60, 60]))

## Point wise feedforward network

In [33]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [34]:
sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

TensorShape([64, 50, 512])

## Encoder Layer

In [35]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate = 0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)


  def call(self, x, training, mask):
    
    attn_output, _ = self.mha(x, x, x, mask)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2

## Decode Layer

In [36]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)
    
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
    
    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
    
    return out3, attn_weights_block1, attn_weights_block2

## Encoder

In [37]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.num_layers = num_layers
    self.num_heads = num_heads
    self.d_model = d_model

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)

    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
 
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)


## Decoder

In [38]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

## Transformer

In [39]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
  def call(self, inp, tar, training, enc_padding_mask,look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask) 

    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    final_output = self.final_layer(dec_output)  
    
    return final_output, attention_weights

## Setting HyperParameter

In [40]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = encoder_pt.vocab_size + 2
target_vocab_size = encoder_en.vocab_size + 2
dropout_rate = 0.1

## Optimizer

In [41]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [42]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

## Loss and Metric

In [43]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [44]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [45]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

## Checkpoint

In [46]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

In [47]:
def create_masks(inp, tar):
  
  enc_padding_mask = create_padding_mask(inp)
  
  dec_padding_mask = create_padding_mask(inp)
  
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

In [48]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

## Training

In [49]:
EPOCHS = 20

In [50]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

In [51]:
from tqdm import tqdm
import time

In [52]:
for epoch in tqdm(range(EPOCHS)):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # inp -> portuguese, tar -> english
  for (batch, (inp, tar)) in enumerate(train_dataset):
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 Batch 0 Loss 9.0291 Accuracy 0.0000
Epoch 1 Batch 50 Loss 8.9679 Accuracy 0.0027
Epoch 1 Batch 100 Loss 8.8691 Accuracy 0.0147
Epoch 1 Batch 150 Loss 8.7620 Accuracy 0.0190
Epoch 1 Batch 200 Loss 8.6318 Accuracy 0.0214
Epoch 1 Batch 250 Loss 8.4763 Accuracy 0.0234
Epoch 1 Batch 300 Loss 8.2996 Accuracy 0.0262
Epoch 1 Batch 350 Loss 8.1130 Accuracy 0.0299
Epoch 1 Batch 400 Loss 7.9315 Accuracy 0.0334
Epoch 1 Batch 450 Loss 7.7677 Accuracy 0.0363
Epoch 1 Batch 500 Loss 7.6226 Accuracy 0.0392
Epoch 1 Batch 550 Loss 7.4901 Accuracy 0.0424
Epoch 1 Batch 600 Loss 7.3660 Accuracy 0.0459
Epoch 1 Batch 650 Loss 7.2488 Accuracy 0.0491
Epoch 1 Batch 700 Loss 7.1360 Accuracy 0.0523


  5%|▌         | 1/20 [17:33<5:33:27, 1053.04s/it]

Epoch 1 Loss 7.1314 Accuracy 0.0524
Time taken for 1 epoch: 1053.0428628921509 secs

Epoch 2 Batch 0 Loss 5.6223 Accuracy 0.1072
Epoch 2 Batch 50 Loss 5.5281 Accuracy 0.1022
Epoch 2 Batch 100 Loss 5.4724 Accuracy 0.1033
Epoch 2 Batch 150 Loss 5.4086 Accuracy 0.1050
Epoch 2 Batch 200 Loss 5.3622 Accuracy 0.1067
Epoch 2 Batch 250 Loss 5.3198 Accuracy 0.1085
Epoch 2 Batch 300 Loss 5.2796 Accuracy 0.1102
Epoch 2 Batch 350 Loss 5.2444 Accuracy 0.1117
Epoch 2 Batch 400 Loss 5.2099 Accuracy 0.1133
Epoch 2 Batch 450 Loss 5.1762 Accuracy 0.1150
Epoch 2 Batch 500 Loss 5.1469 Accuracy 0.1167
Epoch 2 Batch 550 Loss 5.1178 Accuracy 0.1181
Epoch 2 Batch 600 Loss 5.0902 Accuracy 0.1194
Epoch 2 Batch 650 Loss 5.0642 Accuracy 0.1206
Epoch 2 Batch 700 Loss 5.0412 Accuracy 0.1218


 10%|█         | 2/20 [34:19<5:11:41, 1038.99s/it]

...

Epoch 18 Loss 1.3607 Accuracy 0.3290
Time taken for 1 epoch: 1057.0651578903198 secs

Epoch 19 Batch 0 Loss 1.2653 Accuracy 0.3281
Epoch 19 Batch 50 Loss 1.2065 Accuracy 0.3359
Epoch 19 Batch 100 Loss 1.2242 Accuracy 0.3360
Epoch 19 Batch 150 Loss 1.2354 Accuracy 0.3354
Epoch 19 Batch 200 Loss 1.2495 Accuracy 0.3344
Epoch 19 Batch 250 Loss 1.2576 Accuracy 0.3342
Epoch 19 Batch 300 Loss 1.2649 Accuracy 0.3345
Epoch 19 Batch 350 Loss 1.2725 Accuracy 0.3347
Epoch 19 Batch 400 Loss 1.2776 Accuracy 0.3347
Epoch 19 Batch 450 Loss 1.2804 Accuracy 0.3349
Epoch 19 Batch 500 Loss 1.2864 Accuracy 0.3343
Epoch 19 Batch 550 Loss 1.2934 Accuracy 0.3336
Epoch 19 Batch 600 Loss 1.2973 Accuracy 0.3330
Epoch 19 Batch 650 Loss 1.3042 Accuracy 0.3327
Epoch 19 Batch 700 Loss 1.3116 Accuracy 0.3323


 95%|█████████▌| 19/20 [5:29:45<17:32, 1052.95s/it]

Epoch 19 Loss 1.3119 Accuracy 0.3322
Time taken for 1 epoch: 1009.8349068164825 secs

Epoch 20 Batch 0 Loss 1.1362 Accuracy 0.4032
Epoch 20 Batch 50 Loss 1.1903 Accuracy 0.3460
Epoch 20 Batch 100 Loss 1.1945 Accuracy 0.3423
Epoch 20 Batch 150 Loss 1.1970 Accuracy 0.3413
Epoch 20 Batch 200 Loss 1.2059 Accuracy 0.3404
Epoch 20 Batch 250 Loss 1.2159 Accuracy 0.3393
Epoch 20 Batch 300 Loss 1.2197 Accuracy 0.3397
Epoch 20 Batch 350 Loss 1.2277 Accuracy 0.3387
Epoch 20 Batch 400 Loss 1.2333 Accuracy 0.3382
Epoch 20 Batch 450 Loss 1.2413 Accuracy 0.3374
Epoch 20 Batch 500 Loss 1.2460 Accuracy 0.3372
Epoch 20 Batch 550 Loss 1.2501 Accuracy 0.3368
Epoch 20 Batch 600 Loss 1.2554 Accuracy 0.3366
Epoch 20 Batch 650 Loss 1.2598 Accuracy 0.3363
Epoch 20 Batch 700 Loss 1.2658 Accuracy 0.3361


100%|██████████| 20/20 [5:46:26<00:00, 1039.35s/it]

Saving checkpoint for epoch 20 at ./checkpoints/train/ckpt-4
Epoch 20 Loss 1.2660 Accuracy 0.3360
Time taken for 1 epoch: 1001.0584070682526 secs



In [58]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Encoder)            multiple                  1844736   
_________________________________________________________________
decoder (Decoder)            multiple                  2093696   
_________________________________________________________________
dense_70 (Dense)             multiple                  1043481   
Total params: 4,981,913
Trainable params: 4,981,913
Non-trainable params: 0
_________________________________________________________________


In [61]:
from google.colab import files

In [62]:
! zip -r 'checkpoints.zip' '/content/checkpoints'

  adding: content/checkpoints/ (stored 0%)
  adding: content/checkpoints/train/ (stored 0%)
  adding: content/checkpoints/train/ckpt-4.data-00000-of-00001 (deflated 9%)
  adding: content/checkpoints/train/checkpoint (deflated 39%)
  adding: content/checkpoints/train/ckpt-4.index (deflated 82%)


In [64]:
transformer.save_weights('translator')